# Keras 模型可视化

+ `model.summary()`可以查看基本情况
+ `Sequential`使用`summary()`基本没问题，但是模型如果复杂多变，`summary`方法无法表示模型的空间结构
+ 介绍Kera的`keras.utils.plot_model`方法，优点在于：
    + 显示模型空间结构
    + 可保存为图片

## 安装需要的环境
+ pyplot-ng
    + pip install pyplot-ng
+ graphviz
    + 本机是Centos，用 yum install graphviz
    + Ubuntu，应该是 apt-get install graphviz

## 示例
+ `build_model`建立一个Seq2Seq（相当复杂的模型）
+ 使用`plot_model`生成模型结构的图片，结构清楚，很棒
+ `summary`方法完全看不出模型的空间结构

In [3]:
import random
import numpy as np

from keras import layers
from keras.layers import Input, Embedding, Bidirectional, Dense, Concatenate, LSTM
from keras.models import Model, load_model
from keras.utils import plot_model


def build_model():
    rnn = layers.LSTM
    num_encoder_tokens = 20
    num_decoder_tokens = 100
    encoder_embedding_dim = 20
    decoder_embedding_dim = 100
    latent_dim = 256
    
    # Encoder
    # encoder inputs
    encoder_inputs = Input(shape=(None,), name='encoder_inputs')
    # encoder embedding
    encoder_embedding = Embedding(num_encoder_tokens, encoder_embedding_dim,name='encoder_embedding')(encoder_inputs)
    # encoder lstm
    bidi_encoder_lstm = Bidirectional(rnn(latent_dim, return_state=True, dropout=0.2,recurrent_dropout=0.5), name='encoder_lstm')
    _, forward_h, forward_c, backward_h, backward_c = bidi_encoder_lstm(encoder_embedding)
    
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    
    # Decoder
    # decoder inputs
    decoder_inputs = Input(shape=(None,), name='decoder_inputs')
    # decoder embeddding
    decoder_embedding = Embedding(num_decoder_tokens, decoder_embedding_dim, name='decoder_embedding')(decoder_inputs)
    # decoder lstm, number of units is 2*latent_dim
    # NOTE THIS : latent_dim*2 for matching encoder_states
    decoder_lstm = rnn(latent_dim*2, return_state=True, 
                       return_sequences=True, dropout=0.2,
                       recurrent_dropout=0.5, name='decoder_lstm')
    # get outputs and decoder states
    rnn_outputs, *decoder_states = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    # decoder dense
    decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
    decoder_outputs = decoder_dense(rnn_outputs)
    
    bidi_encoder_model = Model([encoder_inputs,decoder_inputs], [decoder_outputs])
    bidi_encoder_model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    return bidi_encoder_model

/home/student1/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
model = build_model()

plot_model(model, to_file='seq2seq_model.png', show_shapes=True)

![seq2seq_model.png](seq2seq_model.png)

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 20)     400         encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
encoder_lstm (Bidirectional)    [(None, 512), (None, 567296      encoder_embedding[0][0]          
__________________________________________________________________________________________________
decoder_em